# Estimativa de volumes de árvores

Modelos de aprendizado de máquina são bastante usados na área da engenharia florestal (mensuração florestal) para, por exemplo, estimar o volume de madeira de árvores sem ser necessário abatê-las.

O processo é feito pela coleta de dados (dados observados) através do abate de algumas árvores, onde sua altura, diâmetro na altura do peito (dap), etc, são medidos de forma exata. Com estes dados, treina-se um modelo de AM que pode estimar o volume de outras árvores da população.

Os modelos, chamados **alométricos**, são usados na área há muitos anos e são baseados em regressão (linear ou não) para encontrar uma equação que descreve os dados. Por exemplo, o modelo de Spurr é dado por:

$$Volume = b_0 + b_1 * dap ^ 2 * H_t$$

Onde **dap** é o diâmetro na altura do peito (1,3 metros), $H_t$ é a altura total. Tem-se vários modelos alométricos, cada um com uma determinada característica, parâmetros, etc. Um modelo de regressão envolve aplicar os dados observados e encontrar $b_0$ e $b_1$ no modelo apresentado, gerando assim uma equação que pode ser usada para prever o volume de outras árvores.

Dado o arquivo **Volumes.csv**, que contém os dados de observação, escolha um modelo de aprendizado de máquina com a melhor estimativa, a partir da estatística de correlação.

**Tarefas**

1. Carregar o arquivo Volumes.csv (http://www.razer.net.br/datasets/Volumes.csv)

2. Eliminar a coluna NR, que só apresenta um número sequencial

3. Criar partição de dados: treinamento 80%, teste 20%

4. Usando o pacote "caret", treinar os modelos: Random Forest (rf), SVM (svmRadial), Redes Neurais (neuralnet) e o modelo alométrico de SPURR

5. O modelo alométrico é dado por: Volume = $b_0$ + $b_1$ * $dap ^2$ * $H_t$

6. Efetue as predições nos dados de teste

7. Crie funções e calcule as seguintes métricas entre a predição e os dados observados

- Coeficiente de determinação: $R^2$

    $$R^2 = 1 - \frac{\sum_{i=1}^n (y_i - \widehat{y_i})^2}{\sum_{i=1}^n (y_i - \overline{y} )^2}$$

    Onde $y_i$ é o valor observado, $\widehat{y_i}$ é o valor predito e $\overline{y}$ é a média dos valores $y_i$ observados. Quanto mais perto de 1 melhor é o modelo;
    
- Erro padrão da estimativa: $S_{yx}$

    $$S_{yx} = \sqrt{\frac{\sum_{i=1}^n (y_i - \widehat{y_i})^2}{n - 2}}$$

    Esta métrica indica erro, portanto quanto mais perto de 0 melhor é o modelo;
    
- $S_{yx}$%:

    $$S_{yx}\% = \frac{S_{xy}}{\overline{y}} * 100$$

    Esta métrica indica porcentagem de erro, portanto quanto mais perto de 0 melhor é o modelo;

8. Escolha o melhor modelo

## Solução

Antes de começarmos precisamos carregar as bibliotecas necessárias para esse exercício:

In [3]:
# install.packages("kernlab")
# install.packages("e1071")
# install.packages("randomForest")
# install.packages("kernlab")
# install.packages("caret")

library("caret")

### Análise de dados

- Carrega os dados

In [4]:
dataset <- read.csv('http://www.razer.net.br/datasets/Volumes.csv', header = TRUE, sep = ';', dec = ',')

In [5]:
head(dataset)

NR,DAP,HT,HP,VOL
1,34.0,27.00,1.80,0.8971441
2,41.5,27.95,2.75,1.6204441
3,29.6,26.35,1.15,0.8008181
4,34.3,27.15,1.95,1.0791682
5,34.5,26.20,1.00,0.9801112
6,29.9,27.10,1.90,0.9067022


- Remove a coluna NR do dataset

In [7]:
dataset <- dataset[-1]

In [8]:
head(dataset)

DAP,HT,HP,VOL
34.0,27.00,1.80,0.8971441
41.5,27.95,2.75,1.6204441
29.6,26.35,1.15,0.8008181
34.3,27.15,1.95,1.0791682
34.5,26.20,1.00,0.9801112
29.9,27.10,1.90,0.9067022


----

### Particionamento

- Particiona os dados em 80% para treinamento e 20% para testes

In [9]:
set.seed(1)
indexes <- createDataPartition(dataset$VOL, p = 0.80, list = FALSE)

In [10]:
dataset.train <- dataset[indexes,]
dataset.test <- dataset[-indexes,]

-----

### Treinamento de modelos

- Random Forest

In [11]:
set.seed(1)
model.rf <- train(VOL ~., data = dataset.train, method = 'rf',
                  trControl = trainControl('cv', number = 10), preProcess = c('center', 'scale'))

note: only 2 unique complexity parameters in default grid. Truncating the grid to 2 .



- Svm

In [12]:
set.seed(1)
model.svm <- train(VOL ~., data = dataset.train, method = 'svmRadial',
                  trControl = trainControl('cv', number = 10), preProcess = c('center', 'scale'))

- Redes neurais

In [13]:
set.seed(1)
model.neural <- train(VOL ~ ., data = dataset.train, method = 'neuralnet', linear.output = TRUE, threshold = 0.1,
                  trControl = trainControl('cv', number = 10), preProcess = c('center', 'scale'))

- Modelo alométrico de SPURR

In [14]:
set.seed(1)
model.alom <- nls(VOL ~ b0 + b1 * DAP ^ 2 * HT, dataset.train, start = list(b0 = 0.5, b1 = 0.5))

-----

### Predições

In [15]:
predictions.rf <- predict(model.rf, dataset.test)

In [16]:
predictions.svm <- predict(model.svm, dataset.test)

In [17]:
predictions.neural <- predict(model.neural, dataset.test)

In [18]:
predictions.alom <- predict(model.alom, dataset.test)

-----

### Métricas

- Cria funções para cálculo de $Rˆ2$, $S_{yx}$ e $S_{yx}\%$

In [19]:
functions.r2 <- function(observations, predictions) {
    return (1 - (sum((observations - predictions) ^ 2) / sum((observations - mean(observations)) ^ 2)))
}

functions.syx <- function(observations, predictions) {
    return (sqrt((sum((observations - predictions) ^ 2) / (length(observations) - 2))))
}

functions.syx_percent <- function(observations, predictions) {
    return (functions.syx(observations, predictions) / mean(observations) * 100)
}

- Métricas para Rando Forest

In [20]:
metrics.rf_r2 <- functions.r2(dataset.test$VOL, predictions.rf)
metrics.rf_syx <- functions.syx(dataset.test$VOL, predictions.rf)
metrics.rf_syx_percent <- functions.syx_percent(dataset.test$VOL, predictions.rf)

- Métricas para SVM

In [22]:
metrics.svm_r2 <- functions.r2(dataset.test$VOL, predictions.svm)
metrics.svm_syx <- functions.syx(dataset.test$VOL, predictions.svm)
metrics.svm_syx_percent <- functions.syx_percent(dataset.test$VOL, predictions.svm)

- Métricas para Neural Network

In [23]:
metrics.neural_r2 <- functions.r2(dataset.test$VOL, predictions.neural)
metrics.neural_syx <- functions.syx(dataset.test$VOL, predictions.neural)
metrics.neural_syx_percent <- functions.syx_percent(dataset.test$VOL, predictions.neural)

- Métricas para o modelo alométrico de Spurr

In [24]:
metrics.alom_r2 <- functions.r2(dataset.test$VOL, predictions.alom)
metrics.alom_syx <- functions.syx(dataset.test$VOL, predictions.alom)
metrics.alom_syx_percent <- functions.syx_percent(dataset.test$VOL, predictions.alom)

-----

### Análise de resultados e conclusão

- Resultado consolidado

In [32]:
data.frame('Random Forest' = c(metrics.rf_r2, metrics.rf_syx, metrics.rf_syx_percent),
           'SVM' = c(metrics.svm_r2, metrics.svm_syx, metrics.svm_syx_percent),
           'Neural Network' = c(metrics.neural_r2, metrics.neural_syx, metrics.neural_syx_percent),
           'Alometric Model' = c(metrics.alom_r2, metrics.alom_syx, metrics.alom_syx_percent), 
           row.names = c('R2', 'Syx','Sxy%'))

,Random.Forest,SVM,Neural.Network,Alometric.Model
R2,0.8159382,0.8471655,0.8666900,0.8328804
Syx,0.1813625,0.1652633,0.1543466,0.1728142
Sxy%,13.0543030,11.8954979,11.1097235,12.4390013


Baseado no nas estatísticas $R^2$, $S_{yx}$ e $S_{yx}\%$, é possível concluir que o melhor modelo treinado é o modelo baseado em redes neurais (Neural Network) pois o mesmo possui a a maior taxa $R^2$ e a menor taxa $S_{xy}$